In [1]:
# DefaultPython (Python 3.11.3) ~/anaconda3/envs/DefaultPython/bin/python
# import session_info
# session_info.show()
# !pipreqsnb . --force
import requests
import copy
import htmlmin
from bs4 import BeautifulSoup
from collections import defaultdict
HEADER = {
    'Accept':'*/*',
    'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'en-GB,en-US;q=0.9,en;q=0.8,zh-CN;q=0.7,zh;q=0.6',
    'Connection':'keep-alive',
    'Referer':'https://dictionary.cambridge.org/dictionary/english-chinese-simplified/',
    'Host':'dictionary.cambridge.org',
    'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
}

COLOURS = {
    'PURPLE':'#7851a9',
    'ORANGE':'#FF8C00',
    'DARKBLUE':'#00308F',
    'LIGHTBLUE':'#007FFF'
}

# Function

In [2]:
def check_none_get_text(bs4_item):
    if bs4_item:
        return bs4_item.get_text()
    else:
        return str()

def return_structured_cambridge_dictionary_word(word):
    url = 'https://dictionary.cambridge.org/dictionary/english-chinese-simplified/'+word
    response = requests.get(url, headers=HEADER)
    response.encoding = 'utf-8'
    res_parsed = BeautifulSoup(response.text, 'html.parser')

    examples=[]
    case = {"case_info":str(),"exp_e":str(),"exp_c":str(),"examples":[]} #case_info:Gold
    phrase = {"form":str(),"phrase_info":str(),"exp_e":str(),"exp_c":str(),"examples":[]}

    dense = {"dense_info":str(), "cases":[], "phrases":[]} #dense_info:Purple
    classs = {"class_info":str(),"pronounciation":str(),"denses":[]}
    single = {"word":str(),"classes":[]}

    single["word"] = check_none_get_text(res_parsed.find("b", {"class": "tb ttn"}))
    for each_class in res_parsed.find_all("div", {"class": "pr entry-body__el"}): # For a class
        classs = {"class_info":str(),"pronounciation":str(),"denses":[]}
        class_header = each_class.find("div", {"class":"pos-header dpos-h"})
        classs["class_info"] = check_none_get_text(class_header.find("div", {"class": "posgram dpos-g hdib lmr-5"})) + ' ' + check_none_get_text(class_header.find("span", {"class":"lab dlab"}))
        classs["pronounciation"] = check_none_get_text(class_header.find("span", {"class":"uk dpron-i"}).find("span", {"class": "pron dpron"}))

        for each_dense in each_class.find_all("div", {"class": "pr dsense"})+each_class.find_all("div", {"class": "pr dsense dsense-noh"}): # For a dense
            dense = {"dense_info":str(), "cases":[], "phrases":[]}
            dense["dense_info"] = check_none_get_text(each_dense.find("h3", {"class": "dsense_h"}))
            main_block = each_dense.find('div', {'class': 'sense-body dsense_b'})
            for each_case in main_block.findChildren("div", {"data-wl-senseid": True} , recursive=False):
                case = {"case_info":str(),"exp_e":str(),"exp_c":str(),"examples":[]}
                case["case_info"] = check_none_get_text(each_case.find("span", {"class":"def-info ddef-info"}))

                case["exp_e"] = check_none_get_text(each_case.find("div", {"class":"def ddef_d db"}))
                second_block = each_case.find("div",{"class":"def-body ddef_b"})
                case["exp_c"] = check_none_get_text(second_block.findChildren("span", {"lang": "zh-Hans"} , recursive=False)[0])
                examples=[]
                for each_example in second_block.findChildren("div", {"class": "examp dexamp"} , recursive=False):
                    examples.append({"exa_e":check_none_get_text(each_example.find("span",{"class":"eg deg"})),
                                     "exa_c":check_none_get_text(each_example.find("span",{"lang":"zh-Hans"}))})
                case["examples"] = copy.deepcopy(examples)
                dense["cases"].append(case)

            for each_phrase in main_block.findChildren("div", {"class": "pr phrase-block dphrase-block lmb-25"} , recursive=False)+main_block.findChildren("div", {"class": "pr phrase-block dphrase-block"} , recursive=False):
                phrase = {"form":str(),"phrase_info":str(),"exp_e":str(),"exp_c":str(),"examples":[]}
                phrase["form"] = check_none_get_text(each_phrase.find("span",{"class":"phrase-title dphrase-title"}))
                phrase["phrase_info"] = check_none_get_text(each_phrase.find("span",{"class":"phrase-info dphrase-info"}))
                second_block = each_phrase.findChildren("div", {"data-wl-senseid": True} , recursive=True)[0]
                phrase["exp_e"] = check_none_get_text(second_block.find("div", {"class":"def ddef_d db"}))
                phrase["exp_c"] = check_none_get_text(second_block.find("span", {"class":"trans dtrans dtrans-se break-cj","lang": "zh-Hans"}))
                examples = []
                for each_example in second_block.find_all("div", {"class": "examp dexamp"}):
                    examples.append({"exa_e":check_none_get_text(each_example.find("span",{"class":"eg deg"})),
                                     "exa_c":check_none_get_text(each_example.find("span",{"class":"trans dtrans dtrans-se hdb break-cj","lang":"zh-Hans"}))})
                phrase["examples"] = copy.deepcopy(examples)
                dense["phrases"].append(phrase)
            classs["denses"].append(dense)
        single["classes"].append(classs)


    print(phrase,'\n')
    print(dense,'\n')
    print(classs,'\n')
    print(single,'\n')

    output = '''<meta charset="utf-8">'''
    output += f'''<span style="font-size:28px;font-weight:bold;">{{{{c1::{single["word"]}}}}}</span><br>'''
    for each_class in single["classes"]:
        output += f"""<span style="font-size:28px;">{each_class["class_info"]} {{{{c1::{each_class["pronounciation"]}}}}}</span>&nbsp"""
        for each_dense in each_class["denses"]:
            output += f"<hr color={COLOURS['PURPLE']} size=3;>"
            if each_dense["dense_info"]:
                output += f"""<span style="{COLOURS['PURPLE']};">{{{{c1::{each_dense["dense_info"]}}}}}</span><br>"""
            for each_case in each_dense["cases"]:
                output += f"<hr color={COLOURS['ORANGE']} size=1;>"
                if each_case["case_info"]:
                    output += f"""<span style="font-size:12px;color:{COLOURS['ORANGE']};">{{{{c1::{each_case["case_info"]}}}}}</span><br>"""
                output += f"""<span style="font-weight:bold;color:{COLOURS['DARKBLUE']};">{{{{c1::{each_case["exp_e"]}}}}}</span><br>"""
                output += f"""<span style="font-weight:bold;color:{COLOURS['LIGHTBLUE']};font-family:KaiTi;">{each_case["exp_c"]}</span><br>"""

                if len(each_case["examples"])>0:
                    output += "<ul>"
                    for each_example in each_case["examples"]:
                        output += "<li>"
                        output += f"""<span style="color:{COLOURS['DARKBLUE']};font-style:italic;">{{{{c1::{each_example["exa_e"]}}}}}</span><br>"""
                        output += f"""<span style="color:{COLOURS['LIGHTBLUE']};font-style:italic;font-family:KaiTi;">{each_example["exa_c"]}</span><br>"""
                        output += "</li>"
                    output += "</ul>"
            for each_phrase in each_dense["phrases"]:
                output += f"<hr color={COLOURS['ORANGE']} size=1;>"
                output += """<div style="background-color:#F0F8FF;padding:5px;">"""

                output += f"""<span style="color:{COLOURS['ORANGE']};">{{{{c1::{each_phrase["form"]} {each_phrase["phrase_info"]}}}}}</span><br>"""
                output += f"""<span style="font-weight:bold;color:{COLOURS['DARKBLUE']};">{{{{c1::{each_phrase["exp_e"]}}}}}</span><br>"""
                if each_phrase["exp_c"] != '':
                    output += f"""<span style="font-weight:bold;color:{COLOURS['LIGHTBLUE']};font-family:KaiTi;">{each_phrase["exp_c"]}</span><br>"""

                if len(each_phrase["examples"])>0:
                    output += "<ul>"
                    for each_example in each_phrase["examples"]:
                        output += "<li>"
                        output += f"""<span style="color:{COLOURS['DARKBLUE']};font-style:italic;">{{{{c1::{each_example["exa_e"]}}}}}</span><br>"""
                        output += f"""<span style="color:{COLOURS['LIGHTBLUE']};font-style:italic;font-family:KaiTi;">{each_example["exa_c"]}</span><br>"""
                        output += "</li>"
                    output += "</ul>"

                output += """</div>"""
    return output

# Under construction
def return_structured_cambridge_dictionary_idiom(word):
    url = 'https://dictionary.cambridge.org/dictionary/english-chinese-simplified/'+word
    response = requests.get(url, headers=HEADER)
    response.encoding = 'utf-8'
    res_parsed = BeautifulSoup(response.text, 'html.parser')

    examples=[]
    case = {"case_info":str(),"exp_e":str(),"exp_c":str(),"examples":[]} #case_info:Gold
    single = {"word":str(),"cases":[]}

    single["word"] = check_none_get_text(res_parsed.find("b", {"class": "tb ttn"}))
    for each_dense in res_parsed.find_all("div", {"class": "pr dsense"})+res_parsed.find_all("div", {"class": "pr dsense dsense-noh"}): # For a dense
        main_block = each_dense.find('div', {'class': 'sense-body dsense_b'})
        for each_case in main_block.findChildren("div", {"class": "def-block ddef_block"}):
            case = {"case_info":str(),"exp_e":str(),"exp_c":str(),"examples":[]}
            case["case_info"] = check_none_get_text(each_case.find("span", {"class":"def-info ddef-info"}))

            case["exp_e"] = check_none_get_text(each_case.find("div", {"class":"def ddef_d db"}))
            second_block = each_case.find("div",{"class":"def-body ddef_b"})
            case["exp_c"]= check_none_get_text(second_block.findChildren("span", {"lang": "zh-Hans"} , recursive=False)[0])
            examples=[]
            for each_example in second_block.findChildren("div", {"class": "examp dexamp"} , recursive=False):
                examples.append({"exa_e":check_none_get_text(each_example.find("span",{"class":"eg deg"})),
                                 "exa_c":check_none_get_text(each_example.find("span",{"lang":"zh-Hans"}))})
            case["examples"] = copy.deepcopy(examples)
            single["cases"].append(case)

    output = '''<meta charset="utf-8">'''
    output += f'''<span style="font-size:28px;font-weight:bold;">{{{{c1::{single["word"]}}}}}</span><br>'''

    for each_case in single["cases"]:
        output += f"<hr color={COLOURS['ORANGE']} size=1;>"
        if each_case["case_info"]:
            output += f"""<span style="font-size:12px;color:#FF8C00;">{{{{c1::{each_case["case_info"]}}}}}</span><br>"""
        output += f"""<span style="font-weight:bold;color:{COLOURS['DARKBLUE']};">{{{{c1::{each_case["exp_e"]}}}}}</span><br>"""
        output += f"""<span style="font-weight:bold;color:{COLOURS['LIGHTBLUE']};font-family:KaiTi;">{each_case["exp_c"]}</span><br>"""

        if len(each_case["examples"])>0:
            output += "<ul>"
            for each_example in each_case["examples"]:
                output += "<li>"
                output += f"""<span style="color:{COLOURS['DARKBLUE']};font-style:italic;">{{{{c1::{each_example["exa_e"]}}}}}</span><br>"""
                output += f"""<span style="color:{COLOURS['LIGHTBLUE']};font-style:italic;font-family:KaiTi;">{each_example["exa_c"]}</span><br>"""
                output += "</li>"
            output += "</ul>"
    return output

def return_structured_cambridge_dictionary(input_text="NULL"):
    if input_text=="NULL":
        print("You must entre a word!")
        return
    if ' '  in input_text:
        print("Idiom service now is under construction, please entre a word instead!")
        # return return_structured_cambridge_dictionary_idiom(input_text.replace(' ','-'))
    else:
        return return_structured_cambridge_dictionary_word(input_text)

In [3]:
input_text = "buckle"
print(htmlmin.minify(return_structured_cambridge_dictionary(input_text)))

{'form': 'buckle under something', 'phrase_info': '', 'exp_e': 'to be defeated by a difficult situation: ', 'exp_c': '', 'examples': [{'exa_e': 'But these were difficult times and a lesser man would have buckled under the strain.', 'exa_c': '但当时是困难时期，脆弱的人肯定会被压力压垮。'}]} 

{'dense_info': 'buckle verb \n            (BE DEFEATED)\n      ', 'cases': [], 'phrases': [{'form': 'buckle under something', 'phrase_info': '', 'exp_e': 'to be defeated by a difficult situation: ', 'exp_c': '', 'examples': [{'exa_e': 'But these were difficult times and a lesser man would have buckled under the strain.', 'exa_c': '但当时是困难时期，脆弱的人肯定会被压力压垮。'}]}]} 

{'class_info': 'verb ', 'pronounciation': '/ˈbʌk.əl/', 'denses': [{'dense_info': 'buckle verb \n            (FASTEN)\n      ', 'cases': [{'case_info': ' [ T or I ]', 'exp_e': 'to fasten or be fastened with a buckle', 'exp_c': '用扣环扣住', 'examples': []}], 'phrases': []}, {'dense_info': 'buckle verb \n            (BEND)\n      ', 'cases': [{'case_info': ' [ T or I ]'

In [ ]:
{'form': 'act/play/star opposite someone', 'phrase_info': '', 'exp_e': 'to act a part in a film or play with someone as a partner: ', 'exp_c': '凯瑟琳‧赫本在许多电影中与亨利‧方达联袂出演。', 'examples': [{'exa_e': 'Katharine Hepburn played opposite Henry Fonda in many films.', 'exa_c': '凯瑟琳‧赫本在许多电影中与亨利‧方达联袂出演。'}]} 
{'form': 'an inveterate liar, gambler, etc.', 'phrase_info': '', 'exp_e': 'someone who does something very often and cannot stop doing it', 'exp_c': '积习难改的说谎者／赌徒（等等）', 'examples': [{'exa_e': "I never trust anything he says - the man's an inveterate liar.", 'exa_c': '我从不相信他说的任何话——这人是个老骗子。'}]} 